In [1]:
import pandas as pd
import ydata_profiling
from ydata_profiling import ProfileReport
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [381]:
# Connect to database
import sqlite3
conn = sqlite3.connect('../Database/sanctions.db') # Remove ../ if used in py code

In [385]:
# Let's check the tables available
pd.read_sql_query("SELECT * FROM sqlite_master", conn)

In [4]:
# Select latest regime file
person_two = pd.read_sql_query('SELECT * from person_two_2023_08_04', conn)

In [384]:
conn.close()

In [115]:
# [1] = "Entity_LogicalId" (Instead of displaying all the count, only displaying count for Entity_LogicalId)

person_two.count()[1]

29166

In [6]:
pers_two_profile = ProfileReport(person_two)

In [386]:
#pers_two_profile

In [113]:
# Based on the data it can be seen 'Entity_LogicalId' is the key and refers to the person
# Hence, Isolate 1 person to establish the data structure 

person_two.query("Entity_LogicalId == 25")

,fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle,Entity_Regulation_Programme,Entity_Regulation_PublicationUrl,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_LogicalId,NameAlias_RegulationLanguage,NameAlias_Remark,NameAlias_Regulation_Type,NameAlias_Regulation_OrganisationType,NameAlias_Regulation_PublicationDate,NameAlias_Regulation_EntryIntoForceDate,NameAlias_Regulation_NumberTitle,NameAlias_Regulation_Programme,NameAlias_Regulation_PublicationUrl,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription,Address_LogicalId,Address_RegulationLanguage,Address_Remark,Address_Regulation_Type,Address_Regulation_OrganisationType,Address_Regulation_PublicationDate,Address_Regulation_EntryIntoForceDate,Address_Regulation_NumberTitle,Address_Regulation_Programme,Address_Regulation_PublicationUrl,BirthDate_BirthDate,BirthDate_Day,BirthDate_Month,BirthDate_Year,BirthDate_YearRangeFrom,BirthDate_YearRangeTo,BirthDate_Circa,BirthDate_CalendarType,BirthDate_ZipCode,BirthDate_Region,BirthDate_Place,BirthDate_City,BirthDate_CountryIso2Code,BirthDate_CountryDescription,BirthDate_LogicalId,BirthDate_RegulationLanguage,BirthDate_Remark,BirthDate_Regulation_Type,BirthDate_Regulation_OrganisationType,BirthDate_Regulation_PublicationDate,BirthDate_Regulation_EntryIntoForceDate,BirthDate_Regulation_NumberTitle,BirthDate_Regulation_Programme,BirthDate_Regulation_PublicationUrl,Identification_Number,Identification_Diplomatic,Identification_KnownExpired,Identification_KnownFalse,Identification_ReportedLost,Identification_RevokedByIssuer,Identification_IssuedBy,Identification_IssuedDate,Identification_ValidFrom,Identification_ValidTo,Identification_LatinNumber,Identification_NameOnDocument,Identification_TypeCode,Identification_TypeDescription,Identification_Region,Identification_CountryIso2Code,Identification_CountryDescription,Identification_LogicalId,Identification_RegulationLanguage,Identification_Remark,Identification_Regulation_Type,Identification_Regulation_OrganisationType,Identification_Regulation_PublicationDate,Identification_Regulation_EntryIntoForceDate,Identification_Regulation_NumberTitle,Identification_Regulation_Programme,Identification_Regulation_PublicationUrl,Citizenship_Region,Citizenship_CountryIso2Code,Citizenship_CountryDescription,Citizenship_LogicalId,Citizenship_RegulationLanguage,Citizenship_Remark,Citizenship_Regulation_Type,Citizenship_Regulation_OrganisationType,Citizenship_Regulation_PublicationDate,Citizenship_Regulation_EntryIntoForceDate,Citizenship_Regulation_NumberTitle,Citizenship_Regulation_Programme,Citizenship_Regulation_PublicationUrl
15,04/08/2023,25,EU.11.43,None,None,None,None,P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,None,None,None,Abid Hamid Mahmud Al-Tikriti,None,M,None,Saddam's Presidential Secretary and Key Advisor,31.0,EN,None,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None
16,04/08/2023,25,EU.11.43,N

#### The above result is consistent with the rest of the data, Each row indicates that at some point data was appended based on "Entity_LogicalId" 

#### Hence we can define the following logic to remove all the None's where there are duplicates

In [15]:
# For each 'fileGenerationDate', 'Entity_LogicalId', concatenate the row in the columns together if it's not None
# This will lead to list being generated for some columns (A function will be defined to later to process this)

person2_mda_1 = person_two.groupby(['fileGenerationDate', 
                                 'Entity_LogicalId'])[
    ['NameAlias_LastName',
     'NameAlias_FirstName',
     'NameAlias_MiddleName',
     'NameAlias_WholeName',
     'NameAlias_NameLanguage',
     'NameAlias_Gender',
     'NameAlias_Title',
     'NameAlias_Function',
     'NameAlias_RegulationLanguage',
     'NameAlias_Remark',
     'Address_City',
     'Address_Street',
     'Address_PoBox',
     'Address_ZipCode',
     'Address_Region',
     'Address_Place',
     'Address_AsAtListingTime',
     'Address_ContactInfo',
     'Address_CountryIso2Code',
     'Address_CountryDescription',
     'Address_RegulationLanguage',
     'Address_Remark',
     'BirthDate_BirthDate',
     'BirthDate_Day',
     'BirthDate_Month',
     'BirthDate_Year',
     'BirthDate_YearRangeFrom',
     'BirthDate_YearRangeTo',
     'BirthDate_Circa',
     'BirthDate_CalendarType',
     'BirthDate_Region',
     'BirthDate_Place',
     'BirthDate_City',
     'BirthDate_CountryIso2Code',
     'BirthDate_CountryDescription',
     'BirthDate_RegulationLanguage',
     'BirthDate_Remark',
     'Identification_Number',
     'Identification_Diplomatic',
     'Identification_KnownExpired',
     'Identification_KnownFalse',
     'Identification_ReportedLost',
     'Identification_RevokedByIssuer',
     'Identification_IssuedBy',
     'Identification_IssuedDate',
     'Identification_ValidFrom',
     'Identification_ValidTo',
     'Identification_LatinNumber',
     'Identification_NameOnDocument',
     'Identification_TypeCode',
     'Identification_TypeDescription',
     'Identification_Region',
     'Identification_CountryIso2Code',
     'Identification_CountryDescription',
     'Identification_RegulationLanguage',
     'Identification_Remark',
     'Citizenship_Region',
     'Citizenship_CountryIso2Code',
     'Citizenship_CountryDescription',
     'Citizenship_RegulationLanguage',
     'Citizenship_Remark']].agg(lambda x: str().join(" ") if x.isnull().all() else x.dropna()).reset_index()

In [112]:
person2_mda_1.query("Entity_LogicalId == 25")

,fileGenerationDate,Entity_LogicalId,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_RegulationLanguage,NameAlias_Remark,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription,Address_RegulationLanguage,Address_Remark,BirthDate_BirthDate,BirthDate_Day,BirthDate_Month,BirthDate_Year,BirthDate_YearRangeFrom,BirthDate_YearRangeTo,BirthDate_Circa,BirthDate_CalendarType,BirthDate_Region,BirthDate_Place,BirthDate_City,BirthDate_CountryIso2Code,BirthDate_CountryDescription,BirthDate_RegulationLanguage,BirthDate_Remark,Identification_Number,Identification_Diplomatic,Identification_KnownExpired,Identification_KnownFalse,Identification_ReportedLost,Identification_RevokedByIssuer,Identification_IssuedBy,Identification_IssuedDate,Identification_ValidFrom,Identification_ValidTo,Identification_LatinNumber,Identification_NameOnDocument,Identification_TypeCode,Identification_TypeDescription,Identification_Region,Identification_CountryIso2Code,Identification_CountryDescription,Identification_RegulationLanguage,Identification_Remark,Citizenship_Region,Citizenship_CountryIso2Code,Citizenship_CountryDescription,Citizenship_RegulationLanguage,Citizenship_Remark
3,04/08/2023,25,,,,"[Abid Hamid Mahmud Al-Tikriti, Abid Hamid Bid ...","[FR, FR, FR]","[M, M, M, M]",Col,[Saddam's Presidential Secretary and Key Advis...,"[EN, EN, EN, EN, EN, EN, EN]",,,,,,,,,,,,,,,,,1957.0,,,YES,GREGORIAN,,,"al-Awja, near Tikrit",00,UNKNOWN,EN,,,,,,,,,,,,,,,,,,,,,,IQ,,EN,


In [20]:
person2_mda_1.count()[1]

4350

In [16]:
# The following columns can be dropped based on duplication
person2_mda_2 = person_two[['fileGenerationDate', 
                            'Entity_LogicalId', 
                            'Entity_EU_ReferenceNumber', 
                            'Entity_UnitedNationId', 
                            'Entity_DesignationDate', 
                            'Entity_DesignationDetails', 
                            'Entity_Remark', 
                            'Entity_SubjectType', 
                            'Entity_SubjectType_ClassificationCode', 
                            'Entity_Regulation_Type', 
                            'Entity_Regulation_OrganisationType', 
                            'Entity_Regulation_PublicationDate', 
                            'Entity_Regulation_EntryIntoForceDate', 
                            'Entity_Regulation_NumberTitle', 
                            'Entity_Regulation_Programme', 
                            'Entity_Regulation_PublicationUrl']].drop_duplicates()

In [116]:
person2_mda_2.query("Entity_LogicalId == 25")

,fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle,Entity_Regulation_Programme,Entity_Regulation_PublicationUrl
15,04/08/2023,25,EU.11.43,None,None,None,None,P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...


In [21]:
person2_mda_2.count()[1]

4353

In [17]:
# based on this count I can see there are 2 records duplicated 
# 3rd record is not a duplicate (search on sanctions.eu for : South Asian Chapter of ISIL)
#person2_mda_2[person2_mda_2['Entity_LogicalId'].duplicated(keep=False)]

In [23]:
# considering it's only 2 records duplicated, we can remove them manually 
person2_mda_2 = person2_mda_2.drop(person2_mda_2[((person2_mda_2.Entity_LogicalId == 547) | 
                                                  (person2_mda_2.Entity_LogicalId == 593)) & 
                                                 (person2_mda_2.Entity_DesignationDate.isnull())].index)

In [343]:
# Join together to get the full data
person_two_mda = person2_mda_2.merge(person2_mda_1, on="Entity_LogicalId", how="left")

In [344]:
person_two_mda.count()[1]

4351

In [345]:
# let's define a function to remove duplicate values from lists in a column 

def parse_values(values):
    if isinstance(values, np.ndarray):
        return pd.unique(values)
    else:
        return values
    
def remove_duplicates_in_list_from_column(df):
    for column in df.columns:
        df[f'{column}'] = df[f'{column}'].apply(parse_values)

In [346]:
remove_duplicates_in_list_from_column(person_two_mda)

In [337]:
# To ensure there's no data missing
person_two_mda.count()[1]

4351

In [347]:
person_two_mda = person_two_mda.drop("fileGenerationDate_y", axis=1)
person_two_mda = person_two_mda.rename(columns={"fileGenerationDate_x" : "fileGenerationDate"})

In [149]:
person_two_mda.query("Entity_LogicalId == 25")

,fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle,Entity_Regulation_Programme,Entity_Regulation_PublicationUrl,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_RegulationLanguage,NameAlias_Remark,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription,Address_RegulationLanguage,Address_Remark,BirthDate_BirthDate,BirthDate_Day,BirthDate_Month,BirthDate_Year,BirthDate_YearRangeFrom,BirthDate_YearRangeTo,BirthDate_Circa,BirthDate_CalendarType,BirthDate_Region,BirthDate_Place,BirthDate_City,BirthDate_CountryIso2Code,BirthDate_CountryDescription,BirthDate_RegulationLanguage,BirthDate_Remark,Identification_Number,Identification_Diplomatic,Identification_KnownExpired,Identification_KnownFalse,Identification_ReportedLost,Identification_RevokedByIssuer,Identification_IssuedBy,Identification_IssuedDate,Identification_ValidFrom,Identification_ValidTo,Identification_LatinNumber,Identification_NameOnDocument,Identification_TypeCode,Identification_TypeDescription,Identification_Region,Identification_CountryIso2Code,Identification_CountryDescription,Identification_RegulationLanguage,Identification_Remark,Citizenship_Region,Citizenship_CountryIso2Code,Citizenship_CountryDescription,Citizenship_RegulationLanguage,Citizenship_Remark
3,04/08/2023,25,EU.11.43,None,None,None,None,P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,,,,"[Abid Hamid Mahmud Al-Tikriti, Abid Hamid Bid ...",[FR],[M],Col,[Saddam's Presidential Secretary and Key Advisor],[EN],,,,,,,,,,,,,,,,,1957.0,,,YES,GREGORIAN,,,"al-Awja, near Tikrit",00,UNKNOWN,EN,,,,,,,,,,,,,,,,,,,,,,IQ,,EN,


In [ ]:
# drop 
# fileGenerationDate,
# fileGenerationDate_y, 

### Set Dtypes

In [123]:
person_two_mda.describe(include="all")

D:\Users\kwame\anaconda3\lib\site-packages\pandas\core\algorithms.py:1028: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  keys, counts = htable.value_count(values, dropna, mask=mask)
D:\Users\kwame\anaconda3\lib\site-packages\pandas\core\algorithms.py:1028: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  keys, counts = htable.value_count(values, dropna, mask=mask)


,fileGenerationDate_x,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle,Entity_Regulation_Programme,Entity_Regulation_PublicationUrl,fileGenerationDate_y,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_RegulationLanguage,NameAlias_Remark,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription,Address_RegulationLanguage,Address_Remark,BirthDate_BirthDate,BirthDate_Day,BirthDate_Month,BirthDate_Year,BirthDate_YearRangeFrom,BirthDate_YearRangeTo,BirthDate_Circa,BirthDate_CalendarType,BirthDate_Region,BirthDate_Place,BirthDate_City,BirthDate_CountryIso2Code,BirthDate_CountryDescription,BirthDate_RegulationLanguage,BirthDate_Remark,Identification_Number,Identification_Diplomatic,Identification_KnownExpired,Identification_KnownFalse,Identification_ReportedLost,Identification_RevokedByIssuer,Identification_IssuedBy,Identification_IssuedDate,Identification_ValidFrom,Identification_ValidTo,Identification_LatinNumber,Identification_NameOnDocument,Identification_TypeCode,Identification_TypeDescription,Identification_Region,Identification_CountryIso2Code,Identification_CountryDescription,Identification_RegulationLanguage,Identification_Remark,Citizenship_Region,Citizenship_CountryIso2Code,Citizenship_CountryDescription,Citizenship_RegulationLanguage,Citizenship_Remark
count,4351,4351.000000,4351,85,3612,159,3967,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351,4351
unique,1,NaN,4350,83,340,86,875,2,2,3,4,293,299,343,33,343,1,2058,1977,1375,4351,1861,5,147,2433,3,668,391,654,54,234,96,52,6,427,171,171,3,236,2235,128,87,258,33,29,33,8,224,290,932,156,154,3,238,819,11,28,4,3,6,19,181,30,111,1,7,242,242,30,124,1,3,335,1,122,1,3,13
top,04/08/2023,NaN,EU.5121.69,QDi.43,2022-02-23,30.7.2020,(Date of UN designation: 2022-02-23),P,person,amendment,council,2022-09-15,2022-09-16,2022/1529 (OJ L239),UKR,https://eur-lex.europa.eu/legal-content/EN/TXT...,04/08/2023,,,,"[Saddam Hussein Al-Tikriti, Abu Ali, Abou Ali]",,[M],,,[EN],,,,,,,,,,,,,,,,,,,,NO,GREGORIAN,,,,,,EN,,,,,,,,,,,,,,,,,,,,,,,,,
freq,4351,NaN,2,2,358,8,358,3490,3490,4145,3812,508,508,508,1803,508,4351,2198,2199,2935,1,1404,2018,3793,1014,3727,3623,3509,3670,4290,4088,4219,4296,3253,3894,3253,3253,3253,4082,1919,1918,1903,1398,4312,4311,2675,2762,4072,4038,2907,1307,1307,2762,3845,3529,3529,3529,3529,3529,3529,4331,4166,4319,4235,4351,4345,3529,3529,4316,3529,4351,3529,3849,4351,2591,4351,2591,4329
mean,NaN,93187.204781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,61966.467400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [348]:
# From the data it can be seen that there some individuals that have duplicated values
# Ideally, these values should be corrected by finding the correct date, however for this project following can be done

list_dates_to_str = ["BirthDate_BirthDate", "Identification_IssuedDate", "NameAlias_Gender", 
                     "Entity_Regulation_EntryIntoForceDate", "NameAlias_NameLanguage"]

for column in list_dates_to_str:
    # if not str, it's list so return the last item
    person_two_mda[column]=[i if isinstance(i, str) else i[-1] for i in person_two_mda[column]]
    # replace empty string " " with NaN values
    person_two_mda[column] = person_two_mda[column].replace(r'^\s*$', np.nan, regex=True)

In [349]:
# Categorical
columns_to_category = ["Entity_SubjectType", "Entity_SubjectType_ClassificationCode", "Entity_Regulation_Type", 
                       "Entity_Regulation_OrganisationType", "NameAlias_Gender"]
for column in columns_to_category:
    person_two_mda[column] = person_two_mda[column].astype("category")    
    

# Dates
date_fields = ["fileGenerationDate", "Entity_DesignationDate", "Entity_Regulation_PublicationDate", 
               "Entity_Regulation_EntryIntoForceDate", "BirthDate_BirthDate", "Identification_IssuedDate"]
person_two_mda[date_fields] = person_two_mda[date_fields].apply(pd.to_datetime)

In [380]:
isinstance(person_two_mda["Entity_Regulation_NumberTitle"].dtype, object)

True

In [382]:
person_two_mda.to_sql(name="person_two_mda", con=conn, if_exists="replace", index=False)

4351